In [2]:
import pandas as pd

df = pd.read_csv('covid_19_clean_complete.csv')
df.Date = pd.to_datetime(df.Date)
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [3]:
total = df.groupby(['Date']).sum().loc[:,['Confirmed','Deaths','Recovered']].reset_index()

total.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1836\3705963557.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total = df.groupby(['Date']).sum().loc[:,['Confirmed','Deaths','Recovered']].reset_index()


,Date,Confirmed,Deaths,Recovered
0,2020-01-22,555,17,28
1,2020-01-23,654,18,30
2,2020-01-24,941,26,36
3,2020-01-25,1434,42,39
4,2020-01-26,2118,56,52


In [ ]:
import fbprophet
from fbprophet.plot import add_changepoints_to_plot

df_prophet= total.rename(columns={'Date': 'ds', 'Confirmed': 'y'})

# Make a future dataframe for X days
m_global = fbprophet.Prophet(changepoint_prior_scale=0.05,changepoint_range=0.95,
                      daily_seasonality=False, 
                      weekly_seasonality=True,
                     mcmc_samples=300)
# Add seasonlity
m_global.add_seasonality(name='monthly', period=30.5, fourier_order=5)

m_global.fit(df_prophet)

# Make predictions
future_global = m_global.make_future_dataframe(periods=30, freq='D')

forecast_global = m_global.predict(future_global)

m_global.plot_components(forecast_global);

In [ ]:
fig =m_global.plot(forecast_global)

In [ ]:
fig = m_global.plot(forecast_global)
a = add_changepoints_to_plot(fig.gca(), m_global, forecast_global)

In [ ]:
# restrict to one country
df_china = df[df['Country/Region']=='Mainland China']
total_china = df_china.groupby(['Date']).sum().loc[:,['Confirmed','Deaths','Recovered']].reset_index()
total_china.head()

In [ ]:
china_prophet= total_china.rename(columns={'Date': 'ds', 'Confirmed': 'y'})

# Make a future dataframe for X days
m_china = fbprophet.Prophet(changepoint_prior_scale=0.05,changepoint_range=0.95,
                      daily_seasonality=False, 
                      weekly_seasonality=True,
                     mcmc_samples=300)
# Add seasonlity
m_china.add_seasonality(name='monthly', period=30.5, fourier_order=5)

m_china.fit(china_prophet)

# Make predictions
future_china = m_china.make_future_dataframe(periods=30, freq='D')

forecast_china = m_china.predict(future_china)

m_china.plot_components(forecast_china);

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))

fig = sns.set_palette('viridis')
fig = sns.lineplot(x='Date', y='Confirmed',data = total, label='Global')
fig = sns.lineplot(x='Date', y='Confirmed',data = total_china, label='China')

sns.despine()
plt.legend()
plt.tight_layout()
plt.xticks(rotation=40)

In [ ]:
print('Percentage of global cases in China: %s' %((total_china.Confirmed.sum()*100)/total.Confirmed.sum()))

In [ ]:
fig = m_china.plot(forecast_china) 

In [ ]:
forecast_china[len(total_china):].loc[:,['ds', 'yhat_lower' ,'yhat_upper', 'yhat']].iloc[:7]

In [ ]:
forecast_global[len(total):].loc[:,['ds', 'yhat_lower' ,'yhat_upper', 'yhat']].iloc[:7]